<a href="https://colab.research.google.com/github/hardeybisey/neural-network/blob/main/traingpt2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install tokenizers
# !wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 48.7 MB/s eta 0:00:00


In [314]:
import torch
import torch.nn as nn
from torch.nn import functional as F
from tokenizers import Tokenizer,PreTokenizedString,NormalizedString, pre_tokenizers, CharBPETokenizer , decoders,  normalizers,ByteLevelBPETokenizer
from tokenizers.models import BPE , WordPiece
from tokenizers.trainers import BpeTrainer
from tokenizers.pre_tokenizers import Whitespace, PreTokenizer
from tokenizers.processors import TemplateProcessing
from tokenizers.normalizers import NFD, Lowercase, StripAccents
torch.manual_seed(45562)

# Read the dataset

In [228]:
text = open('input.txt', 'r', encoding='utf-8').read()
print("length of dataset in characters: ", len(text))
# print(f"\n{text[:200]}")

length of dataset in characters:  1115394


# Build a custom tokenizer with hugging face  tokenizer API

In [ ]:
pre_tokenizer = pre_tokenizers.Sequence([pre_tokenizers.Split("\n",'merged_with_previous'),Whitespace()])

In [ ]:
pre_tokenizer.pre_tokenize_str(text[:50])

[('First Citizen:\n', (0, 15)),
 ('Before we proceed any further, hear', (15, 50))]

In [337]:
class CustomPreTokenizer:
    "A pre_tokenizer that preserves the preserves the new line character"
    def line_split(self, i:int, normalized_string):
        splits = []
        for idx, token in enumerate(normalized_string.split(" ")):
          if "\n" in token:
              idy = token.index("\n")
              subtokens = token.split("\n")
              print(token)
              for i, subtoken in enumerate(subtokens):
                  if i > 0:
                      splits.append(normalized_string[idx][idy])
                  if subtoken:
                      splits.append(normalized_string[idx][i])
          else:
              splits.append(normalized_string[idx])
        return splits

    def pre_tokenize(self, pretok):
        pretok.split(self.line_split)

In [409]:
# c = CustomPreTokenizer()
# c.line_split(0,text[:50])
normalized_string = text[:100]
splits = []
current = 0
for token in normalized_string.split(" "):
  if "\n" in token:
      idy = token.index("\n")
      subtokens = token.split("\n")
      print('norma index', normalized_string.find(token))
      print('current index', current)
      print('token', token)
      for i, subtoken in enumerate(subtokens):
          if i%2 != 0:
              print('new line token',subtoken)
              splits.append(normalized_string[current:current+2])
              current += 2
              splits.append(normalized_string[current:current+len(subtoken)])
              current += len(subtoken)
          else:
              print(subtoken)
              splits.append(normalized_string[current:current+len(subtoken)])
              current += len(subtoken)
  else:
      splits.append(normalized_string[current:current+len(token)])
      current += len(token)

norma index 6
current index 5
token Citizen:
Before
Citizen:
new line token Before
norma index 54
current index 47
token speak.

All:
Speak,
speak.
new line token 
All:
new line token Speak,
norma index 74
current index 67
token speak.

First
speak.
new line token 
First
norma index 88
current index 80
token Citizen:
You
Citizen:
new line token You


In [410]:
'myname is a boy'.find('boy')

12

In [406]:
splits

['First',
 ' Citizen',
 ':\n',
 'Before',
 ' w',
 'e proce',
 'ed ',
 'any furt',
 'her,',
 ' h',
 'ear me',
 ' s',
 '',
 'peak',
 '.\n',
 '\nAll:\n',
 'Speak,',
 ' s',
 '',
 'peak.',
 '\n\nFirst ',
 'Ci',
 'tiz']

In [386]:
'speak.\n\nAll:\nSpeak,'.split("\n")

['speak.', '', 'All:', 'Speak,']

In [383]:
a = text[:500].split(" ")
a#[1].split()

['First',
 'Citizen:\nBefore',
 'we',
 'proceed',
 'any',
 'further,',
 'hear',
 'me',
 'speak.\n\nAll:\nSpeak,',
 'speak.\n\nFirst',
 'Citizen:\nYou',
 'are',
 'all',
 'resolved',
 'rather',
 'to',
 'die',
 'than',
 'to',
 'famish?\n\nAll:\nResolved.',
 'resolved.\n\nFirst',
 'Citizen:\nFirst,',
 'you',
 'know',
 'Caius',
 'Marcius',
 'is',
 'chief',
 'enemy',
 'to',
 'the',
 'people.\n\nAll:\nWe',
 "know't,",
 'we',
 "know't.\n\nFirst",
 'Citizen:\nLet',
 'us',
 'kill',
 'him,',
 'and',
 "we'll",
 'have',
 'corn',
 'at',
 'our',
 'own',
 "price.\nIs't",
 'a',
 'verdict?\n\nAll:\nNo',
 'more',
 'talking',
 "on't;",
 'let',
 'it',
 'be',
 'done:',
 'away,',
 'away!\n\nSecond',
 'Citizen:\nOne',
 'word,',
 'good',
 'citizens.\n\nFirst',
 'Citizen:\nWe',
 'are',
 'accounted',
 'poor']

In [ ]:
w[][]

In [353]:
c.split()

['Citizen:', 'Before']

In [345]:
text[:100].split(" ")[1]

'Citizen:\nBefore'

In [361]:
w = text[:100]

In [342]:
text[:100]

'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'

In [328]:
text[:500]

"First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you know Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us kill him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be done: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor"

In [319]:
tok = ByteLevelBPETokenizer()
# tok.normalizer = normalizers.Sequence([NFD(), StripAccents()])
tok.pre_tokenizer = PreTokenizer.custom(CustomPreTokenizer())
# tok.decoder = Decoder.custom(CustomDecoder())
print(tok.pre_tokenizer.pre_tokenize_str(text[:50]))

Exception: ignored

In [320]:
dir(pre_tokenizers)

['BertPreTokenizer',
 'ByteLevel',
 'CharDelimiterSplit',
 'Digits',
 'Metaspace',
 'PreTokenizer',
 'Punctuation',
 'Sequence',
 'Split',
 'UnicodeScripts',
 'Whitespace',
 'WhitespaceSplit',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 'pre_tokenizers']

In [267]:
sp_tokenizer = Tokenizer(BPE(unk_token="<unk>"))
sp_tokenizer.normalizer = normalizers.Sequence([NFD(), StripAccents()])
sp_tokenizer.pre_tokenizer = PreTokenizer.custom(CustomPreTokenizer())
sp_tokenizer.decoder = decoders.BPEDecoder(suffix="</w>")
class CustomDecoder:
    def decode(self, tokens: List[str]) -> str:
        return "".join(tokens)
# sp_tokenizer.post_processor = TemplateProcessing(single="[CLS] $A [SEP]",
#                                                  pair="[CLS] $A [SEP] $B:1 [SEP]:1",
#                                                  special_tokens=[
#                                                     ("[CLS]", 1),
#                                                     ("[SEP]", 2),
#                                                     ("[NEW-LINE]",3)])

# trainer = BpeTrainer(vocab_size=500, special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"])
# sp_tokenizer.train_from_iterator(text , trainer)

In [270]:
dir(PreTokenizer.custom)

['__call__',
 '__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__name__',
 '__ne__',
 '__new__',
 '__qualname__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__self__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__text_signature__']

In [268]:
print(sp_tokenizer.pre_tokenizer.pre_tokenize_str("Hello"))

Exception: ignored

In [ ]:
toe = CharBPETokenizer()

In [ ]:
tokenizer = Tokenizer.from_pretrained("bert-base-uncased")

In [ ]:
sp_tokenizer.save("sp-tokenizer.json")

In [ ]:
tokenizer = Tokenizer(BPE(unk_token="[UNK]"))
trainer = BpeTrainer(special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"])
tokenizer.pre_tokenizer = Whitespace()
tokenizer.train_from_iterator(text, trainer)
tokenizer.save("sp-tokenizer.json")

In [ ]:
output = tokenizer.encode(text[:50])
output.tokens

['[CLS]',
 'first',
 'citizen',
 ':',
 'before',
 'we',
 'proceed',
 'any',
 'further',
 ',',
 'hear',
 '[SEP]']

In [322]:
https://github.com/huggingface/tokenizers/tree/b24a2fc1781d5da4e6ebcd3ecb5b91edffc0a05f/bindings/python/py_src/tokenizers/implementations

https://huggingface.co/docs/tokenizers/api/trainers

https://huggingface.co/learn/nlp-course/chapter6/8?fw=pt


https://github.com/huggingface/tokenizers

PreTokenize: 永和服装饰品有限公司


Exception: ignored

In [297]:
a = list(jieba.tokenize("永和服装饰品有限公司"))
type(a[0])

tuple